In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import numpy as np
from PIL import Image
import os

# Function to generate a single 32x32 RGB image with unique values for each corner
def generate_image():
    # Initialize a 32x32 image with 3 color channels (RGB)
    image = np.zeros((32, 32, 3), dtype=np.uint8)

    # Generate unique random values for each corner
    dark_value1 = np.random.uniform(0, 0.5)  # Dark value between 0 and 0.5
    dark_value2 = np.random.uniform(0, 0.5)  # Another unique dark value
    light_value1 = np.random.uniform(0.5, 1)  # Light value between 0.5 and 1
    light_value2 = np.random.uniform(0.5, 1)  # Another unique light value

    # Ensure that dark values are different from each other
    while dark_value2 == dark_value1:
        dark_value2 = np.random.uniform(0, 0.5)

    # Ensure that light values are different from each other
    while light_value2 == light_value1:
        light_value2 = np.random.uniform(0.5, 1)

    # Create 16x16 patches with these values
    dark_patch1 = np.full((16, 16, 3), dark_value1 * 255, dtype=np.uint8)
    dark_patch2 = np.full((16, 16, 3), dark_value2 * 255, dtype=np.uint8)
    light_patch1 = np.full((16, 16, 3), light_value1 * 255, dtype=np.uint8)
    light_patch2 = np.full((16, 16, 3), light_value2 * 255, dtype=np.uint8)

    # Place the patches in the appropriate positions
    image[:16, :16] = dark_patch1  # Top-left corner
    image[16:, 16:] = dark_patch2  # Bottom-right corner
    image[:16, 16:] = light_patch1  # Top-right corner
    image[16:, :16] = light_patch2  # Bottom-left corner

    return image

# Function to create a dataset
def create_dataset(num_images, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for i in range(num_images):
        image = generate_image()
        img = Image.fromarray(image, 'RGB')
        img.save(os.path.join(save_dir, f'image_{i}.png'))

# Number of images to generate
num_images = 1000  # Adjust as needed
save_dir = 'synthetic_dataset'

# Generate and save the dataset
create_dataset(num_images, save_dir)

print(f'Dataset generated and saved in {save_dir}')


In [ ]:
#load the images into memory
import numpy as np
from PIL import Image
import os
from sklearn.model_selection import train_test_split

def load_dataset(dataset_dir, img_size=(32, 32)):
    images = []
    labels = []

    for filename in os.listdir(dataset_dir):
        if filename.endswith('.png'):
            # Load image and convert to RGB
            img_path = os.path.join(dataset_dir, filename)
            img = Image.open(img_path).convert('RGB')
            img = img.resize(img_size)  # Resize image
            img_array = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]

            images.append(img_array)

            label = 1
            labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

# Load dataset
dataset_dir = 'synthetic_dataset'
images, labels = load_dataset(dataset_dir)

trainX, testX, trainY, testY = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training images shape: {trainX.shape}")
print(f"Testing images shape: {testX.shape}")
print(f"Training labels shape: {trainY.shape}")
print(f"Testing labels shape: {testY.shape}")


In [ ]:
!pip install keras-vis

In [ ]:
# Plot images from the training dataset
num_images = 49  # Number of images to display
plt.figure(figsize=(10, 10))  # fig size

for i in range(min(num_images, len(trainX))):
    # Define subplot
    plt.subplot(7,7,1+i)
    # Turn off axis
    plt.axis('off')
    # Plot raw pixel data
    plt.imshow(trainX[i])

plt.show()

# Discriminator Model

In [ ]:
#DEFINING DISCRIMINATOR MODEL

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from tensorflow.keras.utils import plot_model
import numpy as np

In [ ]:

#Disriminator model
def define_discriminator(in_shape=(32,32,3)):
  model = Sequential()
  #normal
  model.add(Conv2D(64,(3,3),padding='same',input_shape=in_shape))
  model.add(LeakyReLU(alpha=0.2))
  #downsample
  model.add(Conv2D(128,(3,3),strides=(2,2),padding="same"))
  model.add(LeakyReLU(alpha=0.2))
  #downsample
  model.add(Conv2D(128,(3,3),strides=(2,2),padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  #downsample
  model.add(Conv2D(256,(3,3),strides=(2,2),padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  #classifier
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1,activation='sigmoid'))
  #compile model
  opt = Adam(learning_rate=0.0002,beta_1=0.5)
  model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
  return model


In [ ]:
#define model
model = define_discriminator()
#summarize the model
model.summary()
#Plot the model
plot_model(model, to_file='discriminator_plot.png',show_shapes=True,show_layer_names=True)

In [ ]:
def load_real_samples():
    # Load dataset
    trainX, _ = load_dataset(dataset_dir='synthetic_dataset')
    # Convert from unsigned ints to float
    X = trainX.astype('float32')
    # Scale from [0, 1] to [-1, 1]""
    X = (X - 0.5) / 0.5
    return X


In [ ]:
x_shape = load_real_samples()
x_shape.shape
x_shape[0]

In [ ]:
#select half(64) form real samples
def generate_real_samples(dataset, n_samples):
  #choose random instances
  ix = np.random.randint(0, dataset.shape[0], n_samples)
  #retrieve selected images
  X = dataset[ix]
  #generate 'real' class labels(1)
  y = np.ones((n_samples, 1))
  return X, y

In [ ]:
dataset = load_real_samples()

X_real,y_real = generate_real_samples(dataset, 64)
plt.imshow(X_real[0])


In [ ]:
dataset = load_real_samples()

X_real,y_real = generate_real_samples(dataset, 64)
plt.imshow(X_real[0])


In [ ]:
#generate fake samples with class labels
def generate_fake_samples(n_samples):
  #generate uniform random numbers in [0,1]
  X = np.random.rand(32*32*3*n_samples)
  #update to have the range [-1,1]
  X = -1 + X * 2
  #reshape into a batch of color images
  X = X.reshape((n_samples, 32, 32, 3))
  #generate 'fake' class lables (0)
  y = np.zeros((n_samples, 1))
  return X,y

In [ ]:
X_fake,y_fake = generate_fake_samples(64)
plt.imshow(X_fake[0])

In [ ]:
print(X_fake.shape) #(64, 32, 32, 3)
print(y_fake.shape) #(64, 32, 32, 3)
print(y_fake) #[[0.]..

In [ ]:
#Train The Discriminator model temporaily
def train_discriminator(model, dataset, n_epochs=40, n_batch=128):
  half_batch = int(n_batch / 2)
  #manually enumerate epochs
  for i in range(n_epochs):
    #get randomly selected 'real' samples
    X_real1,y_real1 = generate_real_samples(dataset, half_batch)
    #update discriminator on real samples
    _, real_acc = model.train_on_batch(X_real1, y_real1)
    #generate 'fake' examples
    X_fake1, y_fake1 = generate_fake_samples(half_batch)
    #update discriminator on fake samples
    _, fake_acc = model.train_on_batch(X_fake, y_fake)
    #summarize performance
    print('>%d real=%.0f%% fake=%.0f%%' % (i+1,real_acc*100,fake_acc*100))


In [ ]:
#Example Training

#define the discriminator model
model = define_discriminator()
#load image data
dataset = load_real_samples()
#fit the model
train_discriminator(model, dataset)

#GENERATOR

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from tensorflow.keras.utils import plot_model


In [ ]:
#define the GENERATOR MODEL
def define_generator(latent_dim):
  model = Sequential()
  # foundation for 4x4 image
  n_nodes = 256 * 4 * 4
  model.add(Dense(4096, input_dim = latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((4, 4, 256)))
  # unsample to 8x8
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # unsample to 16x16
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2),padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # unsample to 32x32
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2),padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  #output layer
  model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
  return model

In [ ]:
# define size of latent space (NOISE)
latent_dim = 100
# define the generator model
model = define_generator(latent_dim)
#summarize the model
model.summary()
#plot the model
plot_model(model, to_file='generator_plot.png', show_shapes=True, show_layer_names =True)

In [ ]:
#generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
  #generate points in the latent space
  x_input = np.random.randn(latent_dim * n_samples)
  #reshape into a batch of inputs for the network
  x_input = x_input.reshape(n_samples, latent_dim)
  return x_input

In [ ]:
x_input = generate_latent_points(100, 64)
print(x_input.shape)

In [ ]:
#use the generator to generate n fake samples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
  #generate points in latent space
  x_input = generate_latent_points(latent_dim, n_samples)
  #predict outputs
  X = g_model.predict(x_input)
  #create 'fake' class labels (0)
  y = np.zeros((n_samples, 1))
  return X, y

In [ ]:
#size of latent space
latent_dim = 100
#define the generator model
model = define_generator(latent_dim)
#generate samples
n_samples = 49
X, _ = generate_fake_samples(model, latent_dim, n_samples)
#scale pixel values from [-1,1] to [0,1]
X = (X+1) / 2.0
#Plot the generated samples
for i in range(n_samples):
  #define subplot
  plt.subplot(7, 7 , 1+i)
  #turn off axis labels
  plt.axis('off')
  #plot single image
  plt.imshow(X[i])
#show the figure
plt.show()

In [ ]:
#define the combines generator and discriminator model, for updating the genertor
def define_gan(g_model, d_model):
  #make weights in the discriminator not trainable
  d_model.trainable = False
  #connect them
  model = Sequential()
  #add generator
  model.add(g_model)
  #add the discriminator
  model.add(d_model)
  #comiple model
  opt = Adam(learning_rate=0.0002, beta_1 = 0.5)
  model.compile(loss='binary_crossentropy', optimizer = opt)
  return model

In [ ]:
#size of the latent space
latent_dim = 100
#create the discriminator
d_model = define_discriminator()
#create the generator
g_model = define_generator(latent_dim)
#create the gan
gan_model = define_gan(g_model, d_model)
#summarize gan model
gan_model.summary()
#plot GAN MODEL
plot_model(gan_model,to_file='gan_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=32):
    batch_per_epoch = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    # Manually enumerate epochs
    for i in range(n_epochs):
        # Enumerate batches over the training set
        for j in range(batch_per_epoch):
            # Get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # Update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)

            # Generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # Update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)

            # Prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # Create inverted labels for the fake samples
            y_gan = np.ones((n_batch, 1))

            # Update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)

            # Ensure g_loss is a scalar
            if isinstance(g_loss, list):
                g_loss_value = g_loss[0]
            else:
                g_loss_value = g_loss

            # Summarize loss on this batch
            print(f'>{i+1}, {j+1}/{batch_per_epoch}, d1={d_loss1:.3f}, d2={d_loss2:.3f}, g={g_loss_value:.3f}')

            # Evaluate the model performance, sometimes
            if (i+1) % 2 == 0:
                summarize_performance(i, g_model, d_model, dataset, latent_dim)


In [ ]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
    # Prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # Evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # Prepare fake examples
    X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # Evaluate discriminator performance on fake examples
    _, acc_fake = d_model.evaluate(X_fake, y_fake, verbose=0)
    # Print accuracy
    print(f'>Accuracy real: {acc_real*100:.2f}%, fake: {acc_fake*100:.2f}%')
    # Save plot
    save_plot(X_fake, epoch)
    # Save the generator model as an H5 file
    filename = f'generator_model_{epoch+1:03d}.h5'
    g_model.save(filename)


In [ ]:
#create and save a plot of generated images
def save_plot(examples, epoch, n=7):
  #scale form [-1,1] to [0,1]
  examples = (examples + 1) / 2.0
  #plot images
  for i in range(n * n):
    #define subplot
    plt.subplot(n, n, 1+i)
    #turn off axis
    plt.axis('off')
    #plot raw pixel data
    plt.imshow(examples[i])
  #save plot to file
  filename = 'generated_plot_e%03d.png' % (epoch+1)
  plt.savefig(filename)
  plt.close()

In [ ]:
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=32)

In [ ]:
#example of loading the generator model and generating images
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot as plt

#generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
  #generate points in the latent space
  x_input = randn(latent_dim * n_samples)
  #reshape into a batch of inputs for the network
  x_input = x_input.reshape(n_samples, latent_dim)
  return x_input

#plot the generated images
def create_plot(examples, n):
  #plot images
  for i in range(n * n):
    #define subplot
    plt.subplot(n, n, 1+i)
    #turn off axis
    plt.axis('off')
    #plot raw pixel data
    plt.imshow(examples[i, :, :])
  plt.show()

#load model
model = load_model('generator_model_002.h5')
#generator images
latent_points = generate_latent_points(100, 100)
#generate images
X = model.predict(latent_points)
#scale from [-1,1] to [0,1]
X = (X+1) / 2.0
#plot the result
create_plot(X, 1)

X.shape[0]

In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=2, n_batch=32):
    batch_per_epoch = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    # Manually enumerate epochs
    for i in range(n_epochs):
        # Enumerate batches over the training set
        for j in range(batch_per_epoch):
            # Get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # Update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)

            # Generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # Update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)

            # Prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # Create inverted labels for the fake samples
            y_gan = np.ones((n_batch, 1))

            # Update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)

            # Ensure g_loss is a scalar
            if isinstance(g_loss, list):
                g_loss_value = g_loss[0]
            else:
                g_loss_value = g_loss

            # Summarize loss on this batch
            print(f'>{i+1}, {j+1}/{batch_per_epoch}, d1={d_loss1:.3f}, d2={d_loss2:.3f}, g={g_loss_value:.3f}')

            # Evaluate the model performance, sometimes
            if (i+1) % 10 == 0:
                summarize_performance(i, g_model, d_model, dataset, latent_dim)


In [ ]:
pip install tensorflow keras


In [ ]:
#example of loading the generator model and generating images
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot as plt

#generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
  #generate points in the latent space
  x_input = randn(latent_dim * n_samples)
  #reshape into a batch of inputs for the network
  x_input = x_input.reshape(n_samples, latent_dim)
  return x_input

#plot the generated images
def create_plot(examples, n):
  #plot images
  for i in range(n * n):
    #define subplot
    plt.subplot(n, n, 1+i)
    #turn off axis
    plt.axis('off')
    #plot raw pixel data
    plt.imshow(examples[i, :, :])
  plt.show()

#load model
model = load_model('generator_model_052.h5')
#generator images
latent_points = generate_latent_points(100, 100)
#generate images
X = model.predict(latent_points)
#scale from [-1,1] to [0,1]
X = (X+1) / 2.0
#plot the result
create_plot(X, 1)

X.shape[0]

In [ ]:
#example of loading the generator model and generating images
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot as plt

#generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
  #generate points in the latent space
  x_input = randn(latent_dim * n_samples)
  #reshape into a batch of inputs for the network
  x_input = x_input.reshape(n_samples, latent_dim)
  return x_input

#plot the generated images
def create_plot(examples, n):
  #plot images
  for i in range(n * n):
    #define subplot
    plt.subplot(n, n, 1+i)
    #turn off axis
    plt.axis('off')
    #plot raw pixel data
    plt.imshow(examples[i, :, :])
  plt.show()

#load model
model = load_model('generator_model_002.h5')
#generator images
latent_points = generate_latent_points(100, 100)
#generate images
X = model.predict(latent_points)
#scale from [-1,1] to [0,1]
X = (X+1) / 2.0
#plot the result
create_plot(X, 1)

X.shape[0]